## pydANN ( python deep Artificial Neural Network )

is a free and open source python library to implement the Machine Learning algorithm of neural networks
The network can be as simple as a sinle layer perceptron net or a multi-layer deep neural net.
THe design and modifications of this library is posted [here](https://www.github.com/ShimronAlakkal)


### 1 - Packages

These are some of the most important packages that you're going to need in order to use ***```pydANN```***

- [numpy](www.numpy.org) (or numeric python) is the main package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a library to plot graphs -- which are optional -- in Python.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Custom tools (functions) that are called inside of the activation layer.
To specify the activation function for a layer use `activation_specific = f` with `addHL()`,  where `f` is a list, of length of hidden layers + 1, and each index with a custom function name.
If there is a mismatch in the input activation_specifics, the model is going to auto adjust the activation with the last ones from your input.

In [77]:
def relu(Z):
    return np.maximum(0,Z)

def sigmoid(Z):
    return 1 / 1 + np.exp(-Z)

def leaky_relu(Z):
    return np.maximum(0.1*Z)

def sigmoid_derivative(Z):
    return sigmoid(Z) * ( 1 - sigmoid(Z) ) 

In [142]:
class ann:
    
    def __init__(self,):
        self.hidden_Layers = []
        self.total_layers = []
        self.learning_rate = 0.001
        self.epoch = 0
        self.W = {}
        self.b = {}
        self.Z = {}
        self.A = {}
        self.activation_functions = []
        # self.cost_function 
    
    
    def addHL(self,hl,activations ):
        self.hidden_Layers.clear()
        self.hidden_Layers = hl
        
        # settingf the activations
        self.activation_functions = activations
                
        
    def clear_instance_data():
        self.total_layers.clear()
        self.hidden_Layers.clear()
        self.Z.clear()
        self.W.clear()
        self.b.clear()
            
            
    def register_training_data(self,train_x,train_y):
        self.total_layers.clear()
        self.total_layers.append(train_x.shape[0])
        for i in self.hidden_Layers:
            self.total_layers.append(i)
       
        # network structure
        self.total_layers.append(train_y.shape[0])
        print(f"Network structure update :{self.total_layers}\n feature(s) : {self.total_layers[0]} \n label(s) : {self.total_layers[-1]} ")
        
        
        
    def init_Params(self,verbose = False):
        
        # creating the weights and biases with seed(1)
        np.random.seed(144)
        for i in range(1,len(self.total_layers)):
            
            self.W['W'+str(i)] = np.random.randn(self.total_layers[i],self.total_layers[i-1]) * 0.01
            self.b['b'+str(i)] = np.random.randn(self.total_layers[i],1)
        
        if verbose:
            print('shape of weight(s) initialized : \n ')
            for i in self.W.values():
                print(i.shape)
            print('shape of bias(es) initialized : \n ')
            for i in self.b.values():
                print(i.shape)

    
    
    def forePropagate(self,train_x):
        self.A['A0'] = train_x
        a = self.activation_functions
        
        # populating Z and A with data
        for i in range(1,len(self.total_layers)):
            
            # the formula for fore-propagation is z = W.X + b
            self.Z[ 'Z'+str(i) ] = np.dot( self.W['W'+str(i)] , self.A['A'+str(i-1)] ) + self.b['b'+str(i)]
          
            # populating the activation dictionary with index values
            
            if a[i-1] == 'relu':
                self.A['A'+str(i)] = relu( self.Z['Z'+str(i)] )
            elif a[i-1] == 'sigmoid': 
                self.A['A'+str(i)] = sigmoid( self.Z['Z'+str(i)] )
            elif a[i-1] == 'sigmoid_d':
                self.A['A'+str(i)] = sigmoid_derivative( self.Z['Z'+str(i)] )
            else:
                self.A['A'+str(i)] = leaky_relu( self.Z['Z'+str(i)] )
                
    
    def cost_calc(self):
        pass
    
    def back_prop(self):
        pass

In [134]:
model = ann()

In [135]:
model.addHL([3,2],activations = ['relu','sigmoid','relu'])

In [136]:
model.register_training_data(  np.random.random((3,100)) , np.random.randn(1,100))

Network structure update :[3, 3, 2, 1]
 feature(s) : 3 
 label(s) : 1 


In [137]:
model.init_Params()

In [138]:
model.forePropagate(np.random.random((3,100)))